# Treinamento CNN
Carlos Bravo, Lucas Araujo e Markson Arguello

## Imports

In [47]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
import pickle

## Generate dataset from moves

In [ ]:
def convert_input(board, player_icon, enemy_icon, board_size=8):
    new_board1 = list(
        map(lambda l: list(map(lambda x: 1 if x == player_icon else 0, l)), board))
    new_board2 = list(
        map(lambda l: list(map(lambda x: 1 if x == enemy_icon else 0, l)), board))
    new_board = np.zeros((board_size, board_size, 2), dtype=int)
    new_board[:, :, 0] = new_board1
    new_board[:, :, 1] = new_board2
    return new_board


def check_valid_move(board, move, player_icon, enemy_icon, empty_icon, board_size=8):
    for i in range(-1, 2):
        for j in range(-1, 2):
            if (i != 0 or j != 0):
                # Check if the move is valid
                if (move[0] + i >= 0 and move[0] + i < board_size and move[1] + j >= 0 and move[1] + j < board_size):
                    if (board[move[0] + i][move[1] + j] == enemy_icon):
                        # Check if there is a player icon in the direction
                        k = 2
                        while (move[0] + i*k >= 0 and move[0] + i*k < board_size and move[1] + j*k >= 0 and move[1] + j*k < board_size):
                            if (board[move[0] + i*k][move[1] + j*k] == player_icon):
                                return True
                            if (board[move[0] + i*k][move[1] + j*k] == empty_icon):
                                break
                            k += 1
    return False


def generate_boards(board, moveNum):
    x, y = moveNum // 8, moveNum % 8
    x_flip, y_flip = x, 7 - y
    flipped = np.fliplr(board)
    for i in range(4):
        # Rotate board and move
        yield np.rot90(board, i), x*8 + y
        yield np.rot90(flipped, i), x_flip*8 + y_flip
        x, y = 7-y, x
        x_flip, y_flip = 7-y_flip, x_flip

def create_rows(row):
    rows_set = set()

    # Players icons
    player_icon = 'X'
    enemy_icon = 'O'
    empty_icon = '*'

    # Get moves
    moves_str = df.iloc[row, 0]
    moves = list(map(
        lambda x: (ord(x[0]) - ord('a'), int(x[1]) - 1),
        [moves_str[k:k+2] for k in range(0, len(moves_str), 2)]
    ))

    # Create empty board
    board = [[empty_icon for i in range(8)] for j in range(8)]
    board[3][3] = enemy_icon
    board[4][4] = enemy_icon
    board[3][4] = player_icon
    board[4][3] = player_icon
    
    # For each move
    for move in moves:
        # If move in invalid, change icons back
        if (not check_valid_move(board, move, player_icon, enemy_icon, empty_icon)):
            player_icon, enemy_icon = enemy_icon, player_icon

        # Get move in number format
        move_num = move[0] * 8 + move[1]
        # Add boards to row_array
        for board_rot, move_rot in generate_boards(board, move_num):
            rows_set.add(matrix_to_int(convert_input(
                board_rot, player_icon, enemy_icon), move_rot))
            # print_board_model(convert_input(board_rot, player_icon, enemy_icon), move_rot)
            # print('='*20)

        # Update board
        board[move[0]][move[1]] = player_icon
        for i in range(-1, 2):
            for j in range(-1, 2):
                if (i == 0 and j == 0):
                    continue
                # Check if the move is valid
                if (move[0] + i >= 0 and move[0] + i < 8 and move[1] + j >= 0 and move[1] + j < 8):
                    if (board[move[0] + i][move[1] + j] == enemy_icon):
                        # Check if there is a player icon in the direction
                        k = 2
                        while (move[0] + i*k >= 0 and move[0] + i*k < 8 and move[1] + j*k >= 0 and move[1] + j*k < 8):
                            if (board[move[0] + i*k][move[1] + j*k] == player_icon):
                                for l in range(1, k):
                                    board[move[0] + i*l][move[1] +
                                                         j*l] = player_icon
                                break
                            if (board[move[0] + i*k][move[1] + j*k] == empty_icon):
                                break
                            k += 1
        player_icon, enemy_icon = enemy_icon, player_icon
    return rows_set

def matrix_to_int(board, move):
    return sum([int(board[i][j][k]) * 2**(i + j*8 + k*64) for i in range(len(board)) for j in range(len(board[0])) for k in range(len(board[0][0]))]) * 100 + move

def int_to_matrix(board_int):
    board = np.zeros((8,8,2))
    move = board_int % 100
    board_int = board_int // 100
    for k in range(2):
        for j in range(8):
            for i in range(8):
                board[i][j][k] = board_int % 2
                board_int = board_int // 2
    return board, move

In [ ]:
# Read dataset and create moves set
df = pd.read_csv('data/othello_dataset.csv')
df = df.iloc[:, 2:]

moves_set = set()
for row in range(len(df)):
    if (row % 100 == 0):
        print(f'Processed {row} rows', end='\r')
    moves_set.update(create_rows(row))
print()

with open('data/moves.pickle', 'wb') as f:
    pickle.dump(moves_set, f)

## Create model

In [15]:
def create_model(layers = [64,64,128,128], board_size = 8):
    model = Sequential()
    model.add(Conv2D(layers[0], 3, padding='same', input_shape = (board_size, board_size, 2)))
    for layer in layers[1:]:
        model.add(Conv2D(layer, 3, padding='same'))
    model.add(Flatten())
    model.add(Dense(units = 128))
    model.add(Dense(units = 64, activation = 'sigmoid'))
    return model

In [16]:
def int_to_matrix(board_int):
    board = np.zeros((8,8,2))
    move = board_int % 100
    board_int = board_int // 100
    for k in range(2):
        for j in range(8):
            for i in range(8):
                board[i][j][k] = board_int % 2
                board_int = board_int // 2
    return board, move

In [17]:
model_checkpoint_callback = ModelCheckpoint(
    filepath='ckpt/weights.{epoch:02d}-{val_accuracy:.2f}.hdf5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [5]:
boards = []
moves = []
MAX = 2**12
i = 0

rows = pickle.load(open('data/moves.pickle', 'rb'))
for row in rows:
    if(i == MAX): break
    board, move = int_to_matrix(row)
    boards.append(board)
    moves.append(move)
    i += 1

In [18]:
model = create_model(board_size = 8)
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [19]:
model.fit(
    np.array(boards),
    np.array(moves),
    epochs = 5,
    validation_split = 0.2,
    callbacks = [model_checkpoint_callback],
    shuffle = True
)

Epoch 1/5
103/103 [==============================] - 9s 73ms/step - loss: 3.1429 - accuracy: 0.1694 - val_loss: 2.3305 - val_accuracy: 0.3390
Epoch 2/5
103/103 [==============================] - 8s 74ms/step - loss: 2.4524 - accuracy: 0.2836 - val_loss: 2.1388 - val_accuracy: 0.3866
Epoch 3/5
103/103 [==============================] - 8s 75ms/step - loss: 2.2599 - accuracy: 0.3199 - val_loss: 2.1384 - val_accuracy: 0.3695
Epoch 4/5
103/103 [==============================] - 8s 74ms/step - loss: 2.1501 - accuracy: 0.3526 - val_loss: 2.0572 - val_accuracy: 0.3939
Epoch 5/5
103/103 [==============================] - 8s 79ms/step - loss: 2.0626 - accuracy: 0.3694 - val_loss: 2.1202 - val_accuracy: 0.3841


: 

In [8]:
model.predict(boards[1].reshape(1, 8, 8, 2))

1/1 [==============================] - 0s 375ms/step


array([[ 0.       ,  0.       ,  0.       ,  6.070531 ,  7.376768 ,
         9.703967 ,  8.648658 ,  0.       ,  0.       ,  0.       ,
         5.4359097,  4.1048217,  0.       ,  4.576165 ,  3.409514 ,
         6.401151 ,  0.       ,  0.       ,  0.       ,  9.659093 ,
         0.       ,  6.708563 ,  0.       ,  6.8088517,  0.       ,
         0.       ,  7.061118 ,  0.       ,  0.       , 12.228232 ,
         0.       ,  6.333892 ,  0.       ,  0.       ,  7.1435585,
         0.       ,  0.       ,  7.3747654,  0.       ,  7.452928 ,
         0.       ,  0.       , 14.761819 ,  0.       , 10.396502 ,
         0.       ,  0.       , 11.362385 ,  0.       ,  0.       ,
         0.       , 13.120791 ,  0.       ,  8.995317 ,  8.009962 ,
        17.54161  ,  0.       ,  0.       , 25.056587 ,  0.       ,
         7.015821 ,  7.867293 ,  0.       ,  0.       ]], dtype=float32)